<a href="https://colab.research.google.com/github/Vicks-2019/Pyspark/blob/main/Linear_regression_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,340 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [3]:
# 3. Start Spark Session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
from google.colab import files
files.upload()


Saving Restaurant.csv to Restaurant.csv


{'Restaurant.csv': b'Miscellaneous_Expenses,Food_Innovation_Spend,Advertising,City,Profit\r\n138671.8,167497.2,475918.1,Chicago,202443.83\r\n153151.59,164745.7,448032.53,Mumbai,201974.06\r\n102919.55,155589.51,412068.54,Tokyo,201232.39\r\n120445.85,146520.41,387333.62,Chicago,193083.99\r\n93165.77,144255.34,370302.42,Tokyo,176369.94\r\n101588.71,134024.9,366995.36,Chicago,167173.12\r\n148972.87,136763.46,131850.82,Mumbai,166304.51\r\n147304.06,132446.13,328010.68,Tokyo,165934.6\r\n150492.95,122690.52,315747.29,Chicago,162393.77\r\n110453.17,125482.88,309115.62,Mumbai,159941.96\r\n112368.11,104061.08,233294.95,Tokyo,156303.95\r\n93564.61,102819.96,253878.55,Mumbai,154441.4\r\n129094.38,96011.75,253973.44,Tokyo,151767.52\r\n137269.07,94140.39,256798.93,Mumbai,144489.35\r\n158321.42,122091.24,260646.92,Tokyo,142784.65\r\n124390.84,116671.61,265910.23,Chicago,140099.04\r\n123371.55,80161.11,268480.06,Mumbai,137174.93\r\n146851.58,96805.16,286708.31,Chicago,135552.37\r\n115949.79,93897.16,2

In [6]:

#Loading the Student_Grades_Data.csv file, uploaded in previous step
data = spark.read.csv('Restaurant.csv', header=True, inferSchema=True)

In [7]:

#Taking a look at data type of each column to see what data types inferSchema=TRUE paramter has set for each column
data.printSchema()

root
 |-- Miscellaneous_Expenses: double (nullable = true)
 |-- Food_Innovation_Spend: double (nullable = true)
 |-- Advertising: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Profit: double (nullable = true)



In [8]:
#Display first few rows of data
data.show()

+----------------------+---------------------+-----------+-------+---------+
|Miscellaneous_Expenses|Food_Innovation_Spend|Advertising|   City|   Profit|
+----------------------+---------------------+-----------+-------+---------+
|              138671.8|             167497.2|   475918.1|Chicago|202443.83|
|             153151.59|             164745.7|  448032.53| Mumbai|201974.06|
|             102919.55|            155589.51|  412068.54|  Tokyo|201232.39|
|             120445.85|            146520.41|  387333.62|Chicago|193083.99|
|              93165.77|            144255.34|  370302.42|  Tokyo|176369.94|
|             101588.71|             134024.9|  366995.36|Chicago|167173.12|
|             148972.87|            136763.46|  131850.82| Mumbai|166304.51|
|             147304.06|            132446.13|  328010.68|  Tokyo| 165934.6|
|             150492.95|            122690.52|  315747.29|Chicago|162393.77|
|             110453.17|            125482.88|  309115.62| Mumbai|159941.96|

In [9]:
#Display data types of the data columns.
data.dtypes

[('Miscellaneous_Expenses', 'double'),
 ('Food_Innovation_Spend', 'double'),
 ('Advertising', 'double'),
 ('City', 'string'),
 ('Profit', 'double')]

In [10]:

#Create features storing categorical & numerical variables, omitting the last column
categorical_cols = [item[0] for item in data.dtypes if item[1].startswith('string')]
print(categorical_cols)

numerical_cols = [item[0] for item in data.dtypes if item[1].startswith('int') | item[1].startswith('double')][:-1]
print(numerical_cols)

['City']
['Miscellaneous_Expenses', 'Food_Innovation_Spend', 'Advertising']


In [11]:

#Print number of categorical as well as numerical features.
print(str(len(categorical_cols)) + '  categorical features')
print(str(len(numerical_cols)) + '  numerical features')

1  categorical features
3  numerical features


In [12]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
stages = []
for categoricalCol in categorical_cols:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    OHencoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "_catVec"])
stages += [stringIndexer, OHencoder]
assemblerInputs = [c + "_catVec" for c in categorical_cols] + numerical_cols
Vectassembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [Vectassembler]

In [15]:
import pandas as pd

In [24]:
# Using a Spark MLLib pipeline to apply all the stages of transformation
from pyspark.ml import Pipeline
cols = data.columns
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(data)


data.show()


+--------------------+--------------------+--------------------+--------------------+----------------------+---------------------+-----------+-------+---------+
|            features|            features|            features|            features|Miscellaneous_Expenses|Food_Innovation_Spend|Advertising|   City|   Profit|
+--------------------+--------------------+--------------------+--------------------+----------------------+---------------------+-----------+-------+---------+
|[1.0,0.0,138671.8...|[1.0,0.0,138671.8...|[1.0,0.0,138671.8...|[1.0,0.0,138671.8...|              138671.8|             167497.2|   475918.1|Chicago|202443.83|
|[0.0,1.0,153151.5...|[0.0,1.0,153151.5...|[0.0,1.0,153151.5...|[0.0,1.0,153151.5...|             153151.59|             164745.7|  448032.53| Mumbai|201974.06|
|[0.0,0.0,102919.5...|[0.0,0.0,102919.5...|[0.0,0.0,102919.5...|[0.0,0.0,102919.5...|             102919.55|            155589.51|  412068.54|  Tokyo|201232.39|
|[1.0,0.0,120445.8...|[1.0,0.0,120

In [25]:
finalized_data = data.select("features","Profit")

finalized_data.show()

+--------------------+---------+
|            features|   Profit|
+--------------------+---------+
|[1.0,0.0,138671.8...|202443.83|
|[0.0,1.0,153151.5...|201974.06|
|[0.0,0.0,102919.5...|201232.39|
|[1.0,0.0,120445.8...|193083.99|
|[0.0,0.0,93165.77...|176369.94|
|[1.0,0.0,101588.7...|167173.12|
|[0.0,1.0,148972.8...|166304.51|
|[0.0,0.0,147304.0...| 165934.6|
|[1.0,0.0,150492.9...|162393.77|
|[0.0,1.0,110453.1...|159941.96|
|[0.0,0.0,112368.1...|156303.95|
|[0.0,1.0,93564.61...| 154441.4|
|[0.0,0.0,129094.3...|151767.52|
|[0.0,1.0,137269.0...|144489.35|
|[0.0,0.0,158321.4...|142784.65|
|[1.0,0.0,124390.8...|140099.04|
|[0.0,1.0,123371.5...|137174.93|
|[1.0,0.0,146851.5...|135552.37|
|[0.0,0.0,115949.7...| 134448.9|
|[1.0,0.0,155288.1...|132958.86|
+--------------------+---------+
only showing top 20 rows



In [26]:

#Split the data into training and test model with 70% obs. going in training and 30% in testing
train_dataset, test_dataset = finalized_data.randomSplit([0.7, 0.3])

In [27]:

#Import Linear Regression class called LinearRegression
from pyspark.ml.regression import LinearRegression

In [28]:

#Create the Multiple Linear Regression object named MLR having feature column as features and Label column as Profit
MLR = LinearRegression(featuresCol="features", labelCol="Profit")

In [29]:

#Train the model on the training using fit() method.
model = MLR.fit(train_dataset)

In [30]:
#Predict the Profit on Test Dataset using the evulate method
pred = model.evaluate(test_dataset)

In [31]:
#Show the predicted Grade values along side actual Grade values
pred.predictions.show()

+--------------------+---------+------------------+
|            features|   Profit|        prediction|
+--------------------+---------+------------------+
|[0.0,0.0,102919.5...|201232.39|185478.87038877717|
|[0.0,0.0,107525.0...|118915.99|117861.96290486341|
|[0.0,0.0,112368.1...|156303.95|142948.79197064997|
|[0.0,0.0,115949.7...| 134448.9|136613.98396612404|
|[0.0,0.0,117415.2...|110119.59|109904.44885051208|
|[0.0,0.0,145909.9...|115915.54|122284.82732185622|
|[0.0,1.0,93564.61...| 154441.4|145747.01587615997|
|[0.0,1.0,97963.63...| 81680.49| 84949.71733255673|
|[0.0,1.0,110453.1...|159941.96| 164974.6010750897|
|[0.0,1.0,118757.8...|  24863.4| 65796.52742732184|
|[0.0,1.0,130993.6...|107609.84|114961.48567158589|
|[0.0,1.0,137200.9...| 52741.73| 65772.04092446556|
|[0.0,1.0,156580.1...| 75382.33| 84218.09304114265|
|[1.0,0.0,67721.93...| 91411.06| 75953.40010224747|
|[1.0,0.0,86821.44...|106661.51| 96991.58058337595|
|[1.0,0.0,129156.3...| 79940.98| 72267.36212330495|
|[1.0,0.0,13

In [32]:
#Find out coefficient value
coefficient = model.coefficients
print ("The coefficients of the model are : %a" %coefficient)

The coefficients of the model are : DenseVector([-1481.8266, 4221.882, 0.0465, 0.7661, 0.0195])


In [33]:
#Find out intercept Value
intercept = model.intercept
print ("The Intercept of the model is : %f" %intercept)

The Intercept of the model is : 53438.597281


In [34]:
#Evaluate the model using metric like Mean Absolute Error(MAE), Root Mean Square Error(RMSE) and R-Square
from pyspark.ml.evaluation import RegressionEvaluator
evaluation = RegressionEvaluator(labelCol="Profit", predictionCol="prediction")

# r2 - coefficient of determination
r2 = evaluation.evaluate(pred.predictions, {evaluation.metricName: "r2"})
print("r2: %.3f" %r2)

r2: 0.920


In [35]:

#Create Unlabeled dataset  to contain only feature column
unlabeled_dataset = test_dataset.select('features')

In [36]:
#Display the content of unlabeled_dataset
unlabeled_dataset.show()

+--------------------+
|            features|
+--------------------+
|[0.0,0.0,102919.5...|
|[0.0,0.0,107525.0...|
|[0.0,0.0,112368.1...|
|[0.0,0.0,115949.7...|
|[0.0,0.0,117415.2...|
|[0.0,0.0,145909.9...|
|[0.0,1.0,93564.61...|
|[0.0,1.0,97963.63...|
|[0.0,1.0,110453.1...|
|[0.0,1.0,118757.8...|
|[0.0,1.0,130993.6...|
|[0.0,1.0,137200.9...|
|[0.0,1.0,156580.1...|
|[1.0,0.0,67721.93...|
|[1.0,0.0,86821.44...|
|[1.0,0.0,129156.3...|
|[1.0,0.0,138671.8...|
|[1.0,0.0,146851.5...|
+--------------------+



In [37]:

#Predict the model output for fresh & unseen test data using transform() method
new_predictions = model.transform(unlabeled_dataset)

In [38]:

#Display the new prediction values
new_predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[0.0,0.0,102919.5...|185478.87038877717|
|[0.0,0.0,107525.0...|117861.96290486341|
|[0.0,0.0,112368.1...|142948.79197064997|
|[0.0,0.0,115949.7...|136613.98396612404|
|[0.0,0.0,117415.2...|109904.44885051208|
|[0.0,0.0,145909.9...|122284.82732185622|
|[0.0,1.0,93564.61...|145747.01587615997|
|[0.0,1.0,97963.63...| 84949.71733255673|
|[0.0,1.0,110453.1...| 164974.6010750897|
|[0.0,1.0,118757.8...| 65796.52742732184|
|[0.0,1.0,130993.6...|114961.48567158589|
|[0.0,1.0,137200.9...| 65772.04092446556|
|[0.0,1.0,156580.1...| 84218.09304114265|
|[1.0,0.0,67721.93...| 75953.40010224747|
|[1.0,0.0,86821.44...| 96991.58058337595|
|[1.0,0.0,129156.3...| 72267.36212330495|
|[1.0,0.0,138671.8...| 196031.2280264749|
|[1.0,0.0,146851.5...|138556.85836179502|
+--------------------+------------------+

